# 크롤링 데이터(리뷰) 전처리, 레이블링

In [1]:
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
#from pykospacing import Spacing
from collections import Counter

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [3]:
import json

with open('./naver_횟집_by_metropolitan2.json','r',encoding='utf-8') as f:
    data = json.load(f)

In [51]:
#매장의 개수
len(data)

1775

## 전처리

1. 각 매장 마다의 리뷰 리스트를 나누고
2. null값 확인
3. 한글빼고 삭제 -> pass
4. 중복제거
5. 길이 10이하 텍스트 제거
5. 레이블링

In [52]:
df = pd.DataFrame.from_dict(data, orient='index')
df.reset_index(drop=False, inplace=True)
df

,index,location,review,review_rank,rank
0,청초수물회 송파점,서울 송파구 백제고분로21길 38 1층 102호,[],[],NaN
1,두껍다회선생,서울 성북구 성북로 24 1층,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,...",4.41
2,목포회센타,서울 관악구 남부순환로 1766,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,...",NaN
3,바다회사랑,대전 중구 계룡로874번길 89,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]",3.61
4,은성횟집,서울 종로구 창경궁로 88,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,...",4.27
...,...,...,...,...,...
1770,청연 덕신점,울산 울주군 온산읍 온덕2길 13 1층 청연 덕신점,[],[],4.74
1771,회명품,울산 동구 남진길 63,[],[],NaN
1772,다해횟집,울산 북구 정자1길 23-1 다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]",4.44
1773,더참치,울산 북구 동대중앙로 27 1층 더 참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]",3.43


In [53]:
df.rename(columns={'index': 'store_name'}, inplace=True)
df

,store_name,location,review,review_rank,rank
0,청초수물회 송파점,서울 송파구 백제고분로21길 38 1층 102호,[],[],NaN
1,두껍다회선생,서울 성북구 성북로 24 1층,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,...",4.41
2,목포회센타,서울 관악구 남부순환로 1766,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,...",NaN
3,바다회사랑,대전 중구 계룡로874번길 89,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]",3.61
4,은성횟집,서울 종로구 창경궁로 88,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,...",4.27
...,...,...,...,...,...
1770,청연 덕신점,울산 울주군 온산읍 온덕2길 13 1층 청연 덕신점,[],[],4.74
1771,회명품,울산 동구 남진길 63,[],[],NaN
1772,다해횟집,울산 북구 정자1길 23-1 다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]",4.44
1773,더참치,울산 북구 동대중앙로 27 1층 더 참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]",3.43


In [54]:
df = df.drop(['location', 'rank'],axis=1)
df

,store_name,review,review_rank
0,청초수물회 송파점,[],[]
1,두껍다회선생,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,..."
2,목포회센타,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,..."
3,바다회사랑,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]"
4,은성횟집,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,..."
...,...,...,...
1770,청연 덕신점,[],[]
1771,회명품,[],[]
1772,다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]"
1773,더참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]"


In [55]:
print(sum(df['review_rank'].apply(len) == 0))
zero_review_index = df[df['review'].apply(len) == 0].index
df = df.drop(zero_review_index)

121


In [56]:
df = df.reset_index(drop=True)

In [57]:
df

,store_name,review,review_rank
0,두껍다회선생,[평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음....,"[5, 4, 4.5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4.5,..."
1,목포회센타,[오늘은 광어가 좀 비리네요. 사장님께 말했더니 죄송\n하다고 담엔 바로 얘기하라고...,"[5, 4, 4.5, 4.5, 1, 5, 5, 0.5, 0.5, 4, 4, 4.5,..."
2,바다회사랑,"[저렴한가격에 양 많아요, 좋아요, 굿]","[3.5, 4, 5]"
3,은성횟집,"[광장시장 근본 대구탕,,,\n조금 시끄럽긴 해도 맛은 굳,,,, 약간 짠편이라 안...","[5, 5, 1, 4, 4.5, 3.5, 3, 1, 5, 5, 4, 3, 4, 3,..."
4,횟집울릉도,[#맛있음 #그러나 #계산법이 #괴상함 모듬물회 5인분 포장주문하려하니 포장용기가 ...,"[1, 2, 2, 4, 5, 5, 4, 5, 4.5, 0.5, 5, 3.5, 1, ..."
...,...,...,...
1649,횟집하늘바다,[매번 갈때마다 단골마다 서비스도 거하게 주시고 잘\n먹었습니다 서울에서 오신 손님...,"[5, 5]"
1650,어람횟집,"[맛있어요~^^, ㅎㅎ, 좋아요]","[4, 4, 5]"
1651,다해횟집,"[사장님 친절하시고 음식도맛있고 좋아요~\n서비스도 엄청 좋아요~, 깨끗하고 친절해...","[5, 3.5, 3, 4]"
1652,더참치,"[사장님 부부께서는 참 친절하신데 35,000원짜리를 두명에서 먹을땐 호계치고 그냥...","[3, 0.5, 4]"


In [58]:
df.isnull().sum()
sum(df['review_rank'].apply(len) == 0)

0

In [59]:
#1. 각 매장 마다의 리뷰 리스트를 하나의 리뷰 코퍼스로 만든다
review_list = list()
review_rank_list = list()
for store_idx in df.index:
    review_list.extend(df['review'][store_idx])
    review_rank_list.extend(df['review_rank'][store_idx])
print(review_list[:5])
print(review_rank_list[:5])
    

['평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. 두꺼운 회의 매력에 빠졌달까 ㅎㅎ 우선 생와사비 주는 거 넘나 좋...', '숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가 싸서 이해는하지만\n ㅠ 매운탕 가성비있어서 조아요오 :)', '남자 사장님친절하세요\n여자분은별로...', '근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니 좋네요', '맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용']
['5', '4', '4.5', '4', '5']


In [60]:
print(len(review_list))
len(review_rank_list)

87549


87549

In [61]:
each_review_index = list()
for df_idx in range(len(df)):
    for i in range(len(df.loc[df_idx,'review'])): #각 매장의 리뷰 리스트 개수만큼
        each_review_index.append(df.loc[df_idx,'store_name'])
len(each_review_index)

87549

In [62]:
#데이터 프레임 생성
df = pd.DataFrame({'Store_name' : each_review_index, 'Review': review_list, 'Rank': review_rank_list})
df

,Store_name,Review,Rank
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. ...,5
1,두껍다회선생,숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가...,4
2,두껍다회선생,남자 사장님친절하세요\n여자분은별로...,4.5
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,두껍다회선생,맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...
87544,더참치,참치 맛이 조~~~금 비리대요,4
87545,일품,물회맛있어요~\n매운탕도 같이줘서 굿굿,4.5
87546,일품,맛있고 좋아요,4.5
87547,일품,우럭탕 맛있어여~~^^,5


In [63]:
# null값 확인
df.isnull().sum()

Store_name       0
Review        1820
Rank             0
dtype: int64

In [64]:
# 평점은 있지만 리뷰가 없는 데이터 삭제
df[df['Review'].isnull()]
df = df.dropna()
df = df.reset_index(drop=True)

In [65]:
df

,Store_name,Review,Rank
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음. ...,5
1,두껍다회선생,숙성회맛있ㅅ어요 💕 다만 반반이라는데.. 연어가 \n더 많네요오 ㅠㅠ 연어가 단가가...,4
2,두껍다회선생,남자 사장님친절하세요\n여자분은별로...,4.5
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,두껍다회선생,맛잇어요 존득하고\n활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...
85724,더참치,참치 맛이 조~~~금 비리대요,4
85725,일품,물회맛있어요~\n매운탕도 같이줘서 굿굿,4.5
85726,일품,맛있고 좋아요,4.5
85727,일품,우럭탕 맛있어여~~^^,5


In [66]:
# 한글이 아닌 문자 제거
#여러 공백을 하나의 공백으로 줄인다.
import re
def apply_regular_expreesion(text):
    # 특수문자, 숫자, 구두점 제거
    comp = re.compile('[^ 가-힣]')
    result = comp.sub('', text)
    #여러 공백 하나의 공백으로 변경 
    result = re.sub(' +',' ', result)
    return result

df['Review'] = df['Review'].apply(lambda x : apply_regular_expreesion(str(x)))

In [67]:
df

,Store_name,Review,Rank
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음 두...,5
1,두껍다회선생,숙성회맛있어요 다만 반반이라는데 연어가 더 많네요오 연어가 단가가 싸서 이해는하지만...,4
2,두껍다회선생,남자 사장님친절하세요여자분은별로,4.5
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,두껍다회선생,맛잇어요 존득하고활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...
85724,더참치,참치 맛이 조금 비리대요,4
85725,일품,물회맛있어요매운탕도 같이줘서 굿굿,4.5
85726,일품,맛있고 좋아요,4.5
85727,일품,우럭탕 맛있어여,5


In [68]:
tmp = df[df['Review']=='']
tmp

,Store_name,Review,Rank
110,목포회센타,,4
117,목포회센타,,4
195,은성횟집,,4.5
200,은성횟집,,4
209,은성횟집,,3
...,...,...,...
85496,청년횟집,,5
85608,당월고래전문회타운,,3
85646,정횟집,,5
85653,정횟집,,5


In [74]:
len(np.where(df['Review']== ' ')[0])

31

In [75]:
#빈 문자열 -> NaN 치환
df['Review'].replace('', np.nan, inplace=True)
df['Review'].replace(' ', np.nan, inplace=True)
# null값 확인
print(df.isnull().sum())
df.dropna(inplace=True)
len(df)

Store_name       0
Review        2254
Rank             0
dtype: int64


83475

In [76]:
df.duplicated(['Review']).sum()

23706

In [77]:
#중복 제거
df = df.drop_duplicates(['Review'])
print(df.duplicated(['Review']).sum())
print(len(df))

0
59769


In [78]:
print(df.isnull().sum())

Store_name    0
Review        0
Rank          0
dtype: int64


In [79]:
len(df)

59769

In [80]:
df = df.reset_index(drop=True)

In [81]:
df

,Store_name,Review,Rank
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음 두...,5
1,두껍다회선생,숙성회맛있어요 다만 반반이라는데 연어가 더 많네요오 연어가 단가가 싸서 이해는하지만...,4
2,두껍다회선생,남자 사장님친절하세요여자분은별로,4.5
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,두껍다회선생,맛잇어요 존득하고활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...
59764,더참치,사장님 부부께서는 참 친절하신데 원짜리를 두명에서 먹을땐 호계치고 그냥저냥 만족하며...,3
59765,더참치,엄마 생신이라 평소 가던곳 아닌 더참치 처음 갔어요 만원나왔고 인 열점먹었나테이블당...,0.5
59766,더참치,참치 맛이 조금 비리대요,4
59767,일품,물회맛있어요매운탕도 같이줘서 굿굿,4.5


In [82]:
import pickle
with open('df_tmp.pickle' , 'wb') as f:
    pickle.dump(df,f)

In [83]:
import pickle
with open('df_tmp.pickle' , 'rb') as f:
    df= pickle.load(f)

In [84]:
df

,Store_name,Review,Rank
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음 두...,5
1,두껍다회선생,숙성회맛있어요 다만 반반이라는데 연어가 더 많네요오 연어가 단가가 싸서 이해는하지만...,4
2,두껍다회선생,남자 사장님친절하세요여자분은별로,4.5
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4
4,두껍다회선생,맛잇어요 존득하고활어회보다 저는 이게 더 맛잇는것같아용,5
...,...,...,...
59764,더참치,사장님 부부께서는 참 친절하신데 원짜리를 두명에서 먹을땐 호계치고 그냥저냥 만족하며...,3
59765,더참치,엄마 생신이라 평소 가던곳 아닌 더참치 처음 갔어요 만원나왔고 인 열점먹었나테이블당...,0.5
59766,더참치,참치 맛이 조금 비리대요,4
59767,일품,물회맛있어요매운탕도 같이줘서 굿굿,4.5


In [85]:
df = df.astype({'Rank':'float'})

In [86]:
df.dtypes

Store_name     object
Review         object
Rank          float64
dtype: object

In [87]:
#조건별 데이터 연산
#4~5점 긍정
#0~2점 부정
# 기준점수 3점은 제외
conditions = [
    df['Rank'] >= 3.5,
    df['Rank'] <= 2.5 
]

choices = [
    1,
    0
]
# 레이블링
df['Label'] = np.select(conditions, choices, default= 3)
df[:5]

,Store_name,Review,Rank,Label
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음 두...,5.0,1
1,두껍다회선생,숙성회맛있어요 다만 반반이라는데 연어가 더 많네요오 연어가 단가가 싸서 이해는하지만...,4.0,1
2,두껍다회선생,남자 사장님친절하세요여자분은별로,4.5,1
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4.0,1
4,두껍다회선생,맛잇어요 존득하고활어회보다 저는 이게 더 맛잇는것같아용,5.0,1


In [88]:
df.loc[7,'Label']

3

In [89]:
drop_index = np.where(df['Label'] == 3)

In [90]:
df = df.drop(drop_index[0])

In [91]:
import pickle
with open('df.pickle' , 'wb') as f:
    pickle.dump(df,f)

In [1]:
import pandas as pd
import numpy as np
import pickle
with open('df.pickle' , 'rb') as f:
    df = pickle.load(f)

In [2]:
df

,Store_name,Review,Rank,Label
0,두껍다회선생,평소 두꺼운 회를 별로 안 좋아해서 고민하다가 후기 좋길래 가봤는데 후회 없었음 두...,5.0,1
1,두껍다회선생,숙성회맛있어요 다만 반반이라는데 연어가 더 많네요오 연어가 단가가 싸서 이해는하지만...,4.0,1
2,두껍다회선생,남자 사장님친절하세요여자분은별로,4.5,1
3,두껍다회선생,근처에 들렸다가 밥 먹을때가 안보여서 들어갔어요 가격도 괜찮고 회도 맛있고 깔끔하니...,4.0,1
4,두껍다회선생,맛잇어요 존득하고활어회보다 저는 이게 더 맛잇는것같아용,5.0,1
...,...,...,...,...
59763,다해횟집,주인님 넘넘친절하고 확실하게하시내요 그리고 회는기계로설지안고 손은로직접설어맛이확실히...,4.0,1
59765,더참치,엄마 생신이라 평소 가던곳 아닌 더참치 처음 갔어요 만원나왔고 인 열점먹었나테이블당...,0.5,0
59766,더참치,참치 맛이 조금 비리대요,4.0,1
59767,일품,물회맛있어요매운탕도 같이줘서 굿굿,4.5,1
